In [1]:
!pip install torch-xla -f https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

Looking in links: https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 8.8 MB/s eta 0:00:00


In [2]:
!pip install torch transformers pandas scikit-learn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
sinhala_fake1 = pd.read_csv('/content/drive/MyDrive/RP/sinhala_fake.csv')
sinhala_fake2 = pd.read_excel('/content/drive/MyDrive/RP/sinhala_fake_2.xlsx')
sinhala_real = pd.read_excel('/content/drive/MyDrive/RP/sinhala_real.xlsx')